In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [15]:
#dealing with mnist, 10 output channels requireed minm
from torchsummary import summary

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #28x28 (initially)
        #target rf ~ 28

        self.conv1 = nn.Sequential(  #26, rf = 3
            nn.Conv2d(1, 16, 3, padding=0, stride = 1, bias = False), #input -?(28) OUtput? = (28) RF = (3)
            nn.ReLU(),
            nn.Dropout(0.10)
        )

        self.conv2 = nn.Sequential( #24, #rf = 5
            nn.Conv2d(16, 16, 3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.Dropout(0.10)
        )

        self.conv3 = nn.Sequential( #22, #rf = 7
            nn.Conv2d(16, 32, 3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.Dropout(0.10)
        )

        #TB:  3 > 5> 7 > 9 > 10 > 12 > 12 > 28
        self.tb = nn.Sequential(     #we don't add anythign with tb
            nn.MaxPool2d(2, 2), #size 12  #rf = 8
            nn.Conv2d(32, 16, 1, padding = 0, stride = 1, bias = False) #size = 11 , rf = 8
        )

        self.conv4 = nn.Sequential( #9 , #rf = 12
            nn.Conv2d(16, 16,3, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.Dropout(0.10)
        )

        self.conv5 = nn.Sequential( #9, #rf = 12
            nn.Conv2d(16, 10, 1, padding=0, stride = 1, bias = False),
            nn.ReLU(),
            nn.Dropout(0.10)
        )

        self.conv6 = nn.Sequential( #1, #rf = 28
            nn.Conv2d(10, 10 ,9, padding=0, stride = 1, bias = False),
        )

    def forward(self, x):

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.tb(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

#CB2
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             144
              ReLU-2           [-1, 16, 26, 26]               0
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           2,304
              ReLU-5           [-1, 16, 24, 24]               0
           Dropout-6           [-1, 16, 24, 24]               0
            Conv2d-7           [-1, 32, 22, 22]           4,608
              ReLU-8           [-1, 32, 22, 22]               0
           Dropout-9           [-1, 32, 22, 22]               0
        MaxPool2d-10           [-1, 32, 11, 11]               0
           Conv2d-11           [-1, 16, 11, 11]             512
           Conv2d-12             [-1, 16, 9, 9]           2,304
             ReLU-13             [-1, 16, 9, 9]               0
          Dropout-14             [-1, 1

<ipython-input-15-945aa46fe0f5>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [14]:
from tqdm import tqdm
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        y_pred = model(data)
        pred = y_pred.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

EPOCH: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-13-5d3041baf45e>:65: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.03534857928752899 Batch_id=468 Accuracy=96.05: 100%|██████████| 469/469 [00:17<00:00, 27.17it/s]



Test set: Average loss: 0.0691, Accuracy: 9773/10000 (98%)

EPOCH: 2


Loss=0.01468459889292717 Batch_id=468 Accuracy=98.37: 100%|██████████| 469/469 [00:18<00:00, 25.70it/s]



Test set: Average loss: 0.0519, Accuracy: 9835/10000 (98%)

EPOCH: 3


...


Test set: Average loss: 0.0313, Accuracy: 9904/10000 (99%)

EPOCH: 18


Loss=0.004388894885778427 Batch_id=468 Accuracy=99.46: 100%|██████████| 469/469 [00:17<00:00, 26.60it/s]



Test set: Average loss: 0.0309, Accuracy: 9904/10000 (99%)

EPOCH: 19


Loss=0.029529718682169914 Batch_id=468 Accuracy=99.51: 100%|██████████| 469/469 [00:17<00:00, 27.42it/s]



Test set: Average loss: 0.0303, Accuracy: 9900/10000 (99%)

